In [ ]:
import polars as pl
from pathlib import Path
from sensession.campaign import CampaignProcessor
from IPython.display import display
import hvplot.polars  # noqa: F401

In [ ]:
exp_dir = Path.cwd() / ".." / ".." / "data" / "ax210_averaging"
data = pl.read_parquet(exp_dir / "csi.parquet")
meta = pl.read_parquet(exp_dir / "meta.parquet")

# Equalize data appropriately
proc = (
    CampaignProcessor(csi=data, meta=meta)
    .correct_rssi_by_agc()
    .unwrap()
    .filter("antenna_idxs", 0)
    .detrend_phase()
    .scale_magnitude()
    .equalize_magnitude()
    .equalize_phase()
    .drop_contains("collection_name", "warmup")
)


csi = proc.csi.drop(
    "stream_idxs",
    "antenna_idxs",
    "rssi",
    "antenna_rssi",
    "timestamp",
    "stream_capture_num",
    "rx_antenna_capture_num",
    "capture_num",
    "collection_name",
)
meta = proc.meta

with pl.Config(tbl_cols=-1):
    display(meta.head())
    display(csi.head())

In [ ]:
proc.csi.hvplot.scatter(
    x="subcarrier_idxs", y="csi_abs", groupby=["receiver_name", "schedule_name"]
)

In [ ]:
a = proc.csi.filter(pl.col("receiver_name") == "x310").hvplot.scatter(
    x="subcarrier_idxs", y="csi_abs", groupby=["schedule_name"]
)
b = proc.csi.filter(pl.col("receiver_name") == "ax210").hvplot.scatter(
    x="subcarrier_idxs", y="csi_abs", groupby=["schedule_name"]
)

a * b